## This notebook explores 885 records that were initially hit by the Regex "XX NM dir XXX"
### The output of this notebook is:
#### 1. 821 Lat/Long calculated for FAA Airport Identifier
#### 2. 32  Lat/Long calculated for NAVAID
#### 3. 8 recrods reference runways with no IDENTs, sent to Lex
#### 4. 15 records for FAA airports but with no bearings, sent to Lex
#### 5 . 9 records that do not reference FAA airport nor NAVAID, to be explored later.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import geopy
from geopy.distance import geodesic

In [3]:
# Read in csv file that contains 885 records 
non_std = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/non-std_format_uas_loc.csv')
non_std

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,dateonly,timeonly,DATE,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
0,Aircraft reported UAS activity while entering ...,['3 NM SW of IWA'],IWA,Phoenix-Mesa Gateway,33.307824,-111.655459,KIWA,1/6/2018,1:56,2018-01-06T01:56Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Reported a UAS off the left side at 500 feet w...,['5 NM W of OPF'],OPF,Miami-Opa Locka Exec,25.907417,-80.278222,KOPF,1/12/2018,3:54,2018-01-12T03:54Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Reported a black UAS off the left side of the ...,['5 NM E of SAN'],SAN,San Diego Intl,32.733556,-117.189667,KSAN,1/12/2018,5:58,2018-01-12T05:58Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aircraft reported UAS pass below aircraft whil...,['8 NM ESE of OAK'],OAK,Metro Oakland Intl,37.721250,-122.221139,KOAK,1/15/2018,5:30,2018-01-15T05:30Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Aircraft reported 3 large UAS at 3,000 feet 3....",['5 NM SW of HND'],HND,Henderson Exec,35.972861,-115.134444,KHND,1/21/2018,8:01,2018-01-21T08:01Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,Facility manager observed a UAS from the groun...,['3 NM N of EVV'],EVV,Evansville Rgnl,38.040806,-87.528500,KEVV,8/16/2021,22:02,2021-08-16T22:02Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
881,NWS Seal Beach Base personnel observed a UAS o...,['72 NM SW of SLI'],SLI,Los Alamitos AAF,33.790030,-118.051420,KSLI,8/17/2021,21:14,2021-08-17T21:14Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
882,"PIC reports aircraft struck a multi-colored, 2...",['1 NM E of ORD'],ORD,Chicago O'hare Intl,41.976629,-87.907253,KORD,8/22/2021,23:39,2021-08-22T23:39Z,...,KORD,B,ORD R090 @1DME,250.0,90.0,12.0,NaN,NaN,ENY3961 DEPARTED RWY09C FOR DTW. ONE MILE OFF ...,QA reviewed. ENY3961 departed the ORD Airport ...
883,2120 EDT / 0120 UTC 08/23/2021\r\nENY3961 advi...,['1 NM E of ORD'],ORD,Chicago O'hare Intl,41.976629,-87.907253,KORD,8/23/2021,0:25,2021-08-23T00:25Z,...,KORD,B,1NM ENE ORD RY9C,20.0,360.0,NaN,NaN,NaN,ENY3961 ADVISED C90 SHORTLY AFTER DEPARTING OR...,QA reviewed. All data in linked MOR.


In [4]:
# Find all records that reference airports. 836 records
non_std_air = non_std[non_std['LATITUDE'].notnull()]
non_std_air

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,dateonly,timeonly,DATE,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
0,Aircraft reported UAS activity while entering ...,['3 NM SW of IWA'],IWA,Phoenix-Mesa Gateway,33.307824,-111.655459,KIWA,1/6/2018,1:56,2018-01-06T01:56Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Reported a UAS off the left side at 500 feet w...,['5 NM W of OPF'],OPF,Miami-Opa Locka Exec,25.907417,-80.278222,KOPF,1/12/2018,3:54,2018-01-12T03:54Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Reported a black UAS off the left side of the ...,['5 NM E of SAN'],SAN,San Diego Intl,32.733556,-117.189667,KSAN,1/12/2018,5:58,2018-01-12T05:58Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aircraft reported UAS pass below aircraft whil...,['8 NM ESE of OAK'],OAK,Metro Oakland Intl,37.721250,-122.221139,KOAK,1/15/2018,5:30,2018-01-15T05:30Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Aircraft reported 3 large UAS at 3,000 feet 3....",['5 NM SW of HND'],HND,Henderson Exec,35.972861,-115.134444,KHND,1/21/2018,8:01,2018-01-21T08:01Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,Facility manager observed a UAS from the groun...,['3 NM N of EVV'],EVV,Evansville Rgnl,38.040806,-87.528500,KEVV,8/16/2021,22:02,2021-08-16T22:02Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
881,NWS Seal Beach Base personnel observed a UAS o...,['72 NM SW of SLI'],SLI,Los Alamitos AAF,33.790030,-118.051420,KSLI,8/17/2021,21:14,2021-08-17T21:14Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
882,"PIC reports aircraft struck a multi-colored, 2...",['1 NM E of ORD'],ORD,Chicago O'hare Intl,41.976629,-87.907253,KORD,8/22/2021,23:39,2021-08-22T23:39Z,...,KORD,B,ORD R090 @1DME,250.0,90.0,12.0,NaN,NaN,ENY3961 DEPARTED RWY09C FOR DTW. ONE MILE OFF ...,QA reviewed. ENY3961 departed the ORD Airport ...
883,2120 EDT / 0120 UTC 08/23/2021\r\nENY3961 advi...,['1 NM E of ORD'],ORD,Chicago O'hare Intl,41.976629,-87.907253,KORD,8/23/2021,0:25,2021-08-23T00:25Z,...,KORD,B,1NM ENE ORD RY9C,20.0,360.0,NaN,NaN,NaN,ENY3961 ADVISED C90 SHORTLY AFTER DEPARTING OR...,QA reviewed. All data in linked MOR.


In [5]:
# Extract IDENTs
uas_loc = non_std_air['UAS_LOC'].tolist()
uas_airport = []

for i in range(len(uas_loc)):
    airport = uas_loc[i][-5:-2].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_airport.append(airport)
    else:
        trim_air = airport[-3:]
        uas_airport.append(trim_air)
        
uas_airport_df = pd.DataFrame()
uas_airport_df['IDENT'] = uas_airport
uas_airport_df

,IDENT
0,IWA
1,OPF
2,SAN
3,OAK
4,HND
...,...
831,EVV
832,SLI
833,ORD
834,ORD


In [6]:
# Read is airports dataset
airports = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/airports_cleaned.csv')

In [7]:
# Merge Idents dataframe with airports dataset
uas_air_loc= pd.merge(uas_airport_df, airports, on='IDENT', how='left')
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,IWA,Phoenix-Mesa Gateway,33.307824,-111.655459,KIWA
1,OPF,Miami-Opa Locka Exec,25.907417,-80.278222,KOPF
2,SAN,San Diego Intl,32.733556,-117.189667,KSAN
3,OAK,Metro Oakland Intl,37.721250,-122.221139,KOAK
4,HND,Henderson Exec,35.972861,-115.134444,KHND
...,...,...,...,...,...
831,EVV,Evansville Rgnl,38.040806,-87.528500,KEVV
832,SLI,Los Alamitos AAF,33.790030,-118.051420,KSLI
833,ORD,Chicago O'hare Intl,41.976629,-87.907253,KORD
834,ORD,Chicago O'hare Intl,41.976629,-87.907253,KORD


In [8]:
# Append needed columns
add_col = non_std_air['REMARKS'].tolist()
uas_air_loc['REMARKS'] = add_col
add_col1 = non_std_air['UAS_LOC'].tolist()
uas_air_loc['UAS_LOC'] = add_col1
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UAS_LOC
0,IWA,Phoenix-Mesa Gateway,33.307824,-111.655459,KIWA,Aircraft reported UAS activity while entering ...,['3 NM SW of IWA']
1,OPF,Miami-Opa Locka Exec,25.907417,-80.278222,KOPF,Reported a UAS off the left side at 500 feet w...,['5 NM W of OPF']
2,SAN,San Diego Intl,32.733556,-117.189667,KSAN,Reported a black UAS off the left side of the ...,['5 NM E of SAN']
3,OAK,Metro Oakland Intl,37.721250,-122.221139,KOAK,Aircraft reported UAS pass below aircraft whil...,['8 NM ESE of OAK']
4,HND,Henderson Exec,35.972861,-115.134444,KHND,"Aircraft reported 3 large UAS at 3,000 feet 3....",['5 NM SW of HND']
...,...,...,...,...,...,...,...
831,EVV,Evansville Rgnl,38.040806,-87.528500,KEVV,Facility manager observed a UAS from the groun...,['3 NM N of EVV']
832,SLI,Los Alamitos AAF,33.790030,-118.051420,KSLI,NWS Seal Beach Base personnel observed a UAS o...,['72 NM SW of SLI']
833,ORD,Chicago O'hare Intl,41.976629,-87.907253,KORD,"PIC reports aircraft struck a multi-colored, 2...",['1 NM E of ORD']
834,ORD,Chicago O'hare Intl,41.976629,-87.907253,KORD,2120 EDT / 0120 UTC 08/23/2021\r\nENY3961 advi...,['1 NM E of ORD']


In [9]:
# Create dictionary for bearings and their degrees
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

# Separate distances and bearings into two columns
uas_loc = uas_air_loc['UAS_LOC']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[2:])
    bearings.append(bearing_abrev)

# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(uas_air_loc['REMARKS'])
new_df['UAS_LOC'] = uas_air_loc['UAS_LOC']
new_df['IDENT'] = uas_air_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing
0,Aircraft reported UAS activity while entering ...,['3 NM SW of IWA'],IWA,3,SW
1,Reported a UAS off the left side at 500 feet w...,['5 NM W of OPF'],OPF,5,W
2,Reported a black UAS off the left side of the ...,['5 NM E of SAN'],SAN,5,E
3,Aircraft reported UAS pass below aircraft whil...,['8 NM ESE of OAK'],OAK,8,ESE
4,"Aircraft reported 3 large UAS at 3,000 feet 3....",['5 NM SW of HND'],HND,5,SW
...,...,...,...,...,...
831,Facility manager observed a UAS from the groun...,['3 NM N of EVV'],EVV,3,N
832,NWS Seal Beach Base personnel observed a UAS o...,['72 NM SW of SLI'],SLI,72,SW
833,"PIC reports aircraft struck a multi-colored, 2...",['1 NM E of ORD'],ORD,1,E
834,2120 EDT / 0120 UTC 08/23/2021\r\nENY3961 advi...,['1 NM E of ORD'],ORD,1,E


In [10]:
# Convert distances from NM to KM and bearings to degrees
new_df['Distance'] = pd.to_numeric(new_df['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['Airport_Latitude'] = uas_air_loc['LATITUDE']
new_df['Airport_Longitude'] = uas_air_loc['LONGITUDE']
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude
0,Aircraft reported UAS activity while entering ...,['3 NM SW of IWA'],IWA,3.0,SW,5.556,225,33.307824,-111.655459
1,Reported a UAS off the left side at 500 feet w...,['5 NM W of OPF'],OPF,5.0,W,9.260,270,25.907417,-80.278222
2,Reported a black UAS off the left side of the ...,['5 NM E of SAN'],SAN,5.0,E,9.260,90,32.733556,-117.189667
3,Aircraft reported UAS pass below aircraft whil...,['8 NM ESE of OAK'],OAK,8.0,ESE,14.816,113,37.721250,-122.221139
4,"Aircraft reported 3 large UAS at 3,000 feet 3....",['5 NM SW of HND'],HND,5.0,SW,9.260,225,35.972861,-115.134444
...,...,...,...,...,...,...,...,...,...
831,Facility manager observed a UAS from the groun...,['3 NM N of EVV'],EVV,3.0,N,5.556,0,38.040806,-87.528500
832,NWS Seal Beach Base personnel observed a UAS o...,['72 NM SW of SLI'],SLI,72.0,SW,133.344,225,33.790030,-118.051420
833,"PIC reports aircraft struck a multi-colored, 2...",['1 NM E of ORD'],ORD,1.0,E,1.852,90,41.976629,-87.907253
834,2120 EDT / 0120 UTC 08/23/2021\r\nENY3961 advi...,['1 NM E of ORD'],ORD,1.0,E,1.852,90,41.976629,-87.907253


In [11]:
# Merge, drop and rename columns
new_df2 = pd.merge(new_df, airports, on='IDENT', how='left')
new_df2 = new_df2.drop(['Airport_Latitude', 'Airport_Longitude'], 1)
new_df2 = new_df2.rename({'LONGITUDE': 'Airport_Longitude', 'LATITUDE': 'Airport_Latitude'}, axis=1)
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,Aircraft reported UAS activity while entering ...,['3 NM SW of IWA'],IWA,3.0,SW,5.556,225,Phoenix-Mesa Gateway,33.307824,-111.655459,KIWA
1,Reported a UAS off the left side at 500 feet w...,['5 NM W of OPF'],OPF,5.0,W,9.260,270,Miami-Opa Locka Exec,25.907417,-80.278222,KOPF
2,Reported a black UAS off the left side of the ...,['5 NM E of SAN'],SAN,5.0,E,9.260,90,San Diego Intl,32.733556,-117.189667,KSAN
3,Aircraft reported UAS pass below aircraft whil...,['8 NM ESE of OAK'],OAK,8.0,ESE,14.816,113,Metro Oakland Intl,37.721250,-122.221139,KOAK
4,"Aircraft reported 3 large UAS at 3,000 feet 3....",['5 NM SW of HND'],HND,5.0,SW,9.260,225,Henderson Exec,35.972861,-115.134444,KHND
...,...,...,...,...,...,...,...,...,...,...,...
831,Facility manager observed a UAS from the groun...,['3 NM N of EVV'],EVV,3.0,N,5.556,0,Evansville Rgnl,38.040806,-87.528500,KEVV
832,NWS Seal Beach Base personnel observed a UAS o...,['72 NM SW of SLI'],SLI,72.0,SW,133.344,225,Los Alamitos AAF,33.790030,-118.051420,KSLI
833,"PIC reports aircraft struck a multi-colored, 2...",['1 NM E of ORD'],ORD,1.0,E,1.852,90,Chicago O'hare Intl,41.976629,-87.907253,KORD
834,2120 EDT / 0120 UTC 08/23/2021\r\nENY3961 advi...,['1 NM E of ORD'],ORD,1.0,E,1.852,90,Chicago O'hare Intl,41.976629,-87.907253,KORD


In [12]:
# Find records with no bearings or distance
none = new_df2[(new_df2['Bearing'] == '') | (new_df2['Distance'].isnull())]
none

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
78,Reported a UAS at 400 feet over the sea wall w...,['5 NM of NGP'],NGP,5.0,,9.260,,Corpus Christi NAS/Truax Fld,27.692701,-97.290377,KNGP
90,"1757: Reported a white, rotorcraft type UAS 6 ...",['6 N of RIC'],RIC,6.0,,11.112,,Richmond Intl,37.505181,-77.319739,KRIC
139,"DCA-1: DCA070008 (7 N of ADW), MUSL7 reported ...",['7 N of ADW'],ADW,7.0,,12.964,,Joint Base Andrews,38.810826,-76.867046,KADW
286,Reported a 3 foot in diameter white quad copte...,['4 N of EWR'],EWR,4.0,,7.408,,Newark Liberty Intl,40.692480,-74.168687,KEWR
324,"Reported a silver UAS while S bound at 1,200 f...",['18 N of AVP'],AVP,18.0,,33.336,,Wilkes-Barre/Scranton Intl,41.338472,-75.723389,KAVP
326,"Reported a UAS at 10,500 feet at the DCA230020...",['7 N of NYG'],NYG,7.0,,12.964,,Quantico MCAF /Turner Fld,38.503625,-77.305033,KNYG
327,"Reported a red UAS while NNW bound at 4,000 fe...",['2 N of LGA'],LGA,2.0,,3.704,,Laguardia,40.777250,-73.872611,KLGA
334,"Reported a UAS at same altitude of 2,000 feet ...",['8 N of LGA'],LGA,8.0,,14.816,,Laguardia,40.777250,-73.872611,KLGA
370,Reported a UAS within 200 yards below aircraft...,['5 N of HSV'],HSV,5.0,,9.260,,Huntsville Intl-Carl T Jones Fld,34.637197,-86.775054,KHSV
417,Reported a UAS 500 feet above while NW bound a...,['8 N of STL'],STL,8.0,,14.816,,St Louis Lambert Intl,38.748697,-90.370029,KSTL


In [12]:
# Export these records to csv file and send them to Lex
none= pd.merge(none, non_std_air, on='REMARKS', how='left')

none.to_csv('Non-std-airports-15NoBearing.csv', index = False)

In [13]:
# Find records that have bearings and distances
new_df2 = new_df2[(new_df2['Bearing'] != '')]
new_df2 = new_df2[(new_df2['Distance'].notnull())] 
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,Aircraft reported UAS activity while entering ...,['3 NM SW of IWA'],IWA,3.0,SW,5.556,225,Phoenix-Mesa Gateway,33.307824,-111.655459,KIWA
1,Reported a UAS off the left side at 500 feet w...,['5 NM W of OPF'],OPF,5.0,W,9.260,270,Miami-Opa Locka Exec,25.907417,-80.278222,KOPF
2,Reported a black UAS off the left side of the ...,['5 NM E of SAN'],SAN,5.0,E,9.260,90,San Diego Intl,32.733556,-117.189667,KSAN
3,Aircraft reported UAS pass below aircraft whil...,['8 NM ESE of OAK'],OAK,8.0,ESE,14.816,113,Metro Oakland Intl,37.721250,-122.221139,KOAK
4,"Aircraft reported 3 large UAS at 3,000 feet 3....",['5 NM SW of HND'],HND,5.0,SW,9.260,225,Henderson Exec,35.972861,-115.134444,KHND
...,...,...,...,...,...,...,...,...,...,...,...
831,Facility manager observed a UAS from the groun...,['3 NM N of EVV'],EVV,3.0,N,5.556,0,Evansville Rgnl,38.040806,-87.528500,KEVV
832,NWS Seal Beach Base personnel observed a UAS o...,['72 NM SW of SLI'],SLI,72.0,SW,133.344,225,Los Alamitos AAF,33.790030,-118.051420,KSLI
833,"PIC reports aircraft struck a multi-colored, 2...",['1 NM E of ORD'],ORD,1.0,E,1.852,90,Chicago O'hare Intl,41.976629,-87.907253,KORD
834,2120 EDT / 0120 UTC 08/23/2021\r\nENY3961 advi...,['1 NM E of ORD'],ORD,1.0,E,1.852,90,Chicago O'hare Intl,41.976629,-87.907253,KORD


In [33]:
new_df2.to_csv('check.csv', index = False)
new_df2 = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/check.csv')

In [34]:
# Calculate UAS lat/long
uas_lat = []
uas_long = []

for i in range(len(new_df2)):
    lat_airport = pd.to_numeric(new_df2['Airport_Latitude'][i])
    long_airport = pd.to_numeric(new_df2['Airport_Longitude'][i])
    b = pd.to_numeric(new_df2['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df2['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
new_df2['UAS_Latitude'] = uas_lat
new_df2['UAS_Longitude'] = uas_long
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID,UAS_Latitude,UAS_Longitude
0,Aircraft reported UAS activity while entering ...,['3 NM SW of IWA'],IWA,3.0,SW,5.556,225,Phoenix-Mesa Gateway,33.307824,-111.655459,KIWA,33.272394,-111.697628
1,Reported a UAS off the left side at 500 feet w...,['5 NM W of OPF'],OPF,5.0,W,9.260,270,Miami-Opa Locka Exec,25.907417,-80.278222,KOPF,25.907387,-80.370641
2,Reported a black UAS off the left side of the ...,['5 NM E of SAN'],SAN,5.0,E,9.260,90,San Diego Intl,32.733556,-117.189667,KSAN,32.733517,-117.090876
3,Aircraft reported UAS pass below aircraft whil...,['8 NM ESE of OAK'],OAK,8.0,ESE,14.816,113,Metro Oakland Intl,37.721250,-122.221139,KOAK,37.668990,-122.066556
4,"Aircraft reported 3 large UAS at 3,000 feet 3....",['5 NM SW of HND'],HND,5.0,SW,9.260,225,Henderson Exec,35.972861,-115.134444,KHND,35.913828,-115.206987
...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,Facility manager observed a UAS from the groun...,['3 NM N of EVV'],EVV,3.0,N,5.556,0,Evansville Rgnl,38.040806,-87.528500,KEVV,38.090861,-87.528500
817,NWS Seal Beach Base personnel observed a UAS o...,['72 NM SW of SLI'],SLI,72.0,SW,133.344,225,Los Alamitos AAF,33.790030,-118.051420,KSLI,32.935774,-119.059606
818,"PIC reports aircraft struck a multi-colored, 2...",['1 NM E of ORD'],ORD,1.0,E,1.852,90,Chicago O'hare Intl,41.976629,-87.907253,KORD,41.976627,-87.884908
819,2120 EDT / 0120 UTC 08/23/2021\r\nENY3961 advi...,['1 NM E of ORD'],ORD,1.0,E,1.852,90,Chicago O'hare Intl,41.976629,-87.907253,KORD,41.976627,-87.884908


In [35]:
# Export records to csv file
new_df2 = new_df2.drop(['Distance','Bearing','Distance_Kilometers','Bearing_Degrees','ICAO_ID'], 1)
new_df2 = new_df2.rename({'Airport_Longitude': 'Origin_Longitude', 'Airport_Latitude': 'Origin_Latitude'}, axis=1)
new_df2 = pd.merge(new_df2, non_std, on='REMARKS', how='left')
new_df2 = new_df2.drop(['IDENT_y','NAME_y','UAS_LOC_y'], 1)
new_df2 = new_df2.rename({'IDENT_x': 'IDENT', 'NAME_x': 'NAME', 'UAS_LOC_x':'UAS_LOC'}, axis=1)
new_df2.to_csv('COMPLETED-Non-std-821Airports.csv', index = False)

In [ ]:
#######################################################################################################################

In [14]:
# Find all the null values and put them in new dataframe
non_std1 = non_std[non_std['NAME'].isnull()]
non_std1 = non_std1.drop(['NAME', 'LATITUDE', 'LONGITUDE', 'ICAO_ID'], axis=1)

non_std1 # There are 49 records with no airports

,REMARKS,UAS_LOC,IDENT,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
75,FACSFAC VACAPES reports PIC reported a silver ...,['6 NM E of RBX'],RBX,5/4/2018,0:06,2018-05-04T00:06Z,PARTY31,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Reported a UAS 500 feet below aircraft while 3...,['3 NM E of OSI'],OSI,5/14/2018,5:46,2018-05-14T05:46Z,ASA330,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,Reported a silver and white fixed wing UAS off...,['11 NM SW of OSI'],OSI,5/14/2018,5:53,2018-05-14T05:53Z,N2809E,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,PIC reported a black rotary UAS 50 feet below ...,['2 NM SW of PQF'],PQF,5/30/2018,0:31,2018-05-30T00:31Z,XAZTK,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,After departing RWY 10 PIC reported a UAS belo...,['3 NM NE of TIST'],TIST,5/30/2018,5:00,2018-05-30T05:00Z,KAP249,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,PIC reported a black and brown UAS 1-meter in ...,['10 NM NW of CMK'],CMK,6/4/2018,4:51,2018-06-04T04:51Z,DLH403,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,Forest Service Firehelicopter reported UASflew...,['7 NM NNE of DBL'],DBL,6/11/2018,5:53,2018-06-11T05:53Z,NaN,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,Reported a drone hovering at 600-700 feet 7 NM...,['7 NM N of BAL'],BAL,6/17/2018,8:25,2018-06-17T08:25Z,FOXX2,NCRCC,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,LUKE AFB called and advised of a UAS sighting ...,['20 NM SW of GBN'],GBN,6/19/2018,22:24,2018-06-19T22:24Z,NaN,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222,"Reported NM AC with a black with red UAS, 4 to...",['26 NM SE of JLI'],JLI,6/22/2018,23:22,2018-06-22T23:22Z,RANGR43,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Read in the NAVAID dataset
navaid = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/NAVAID_Cleaned.csv')
navaid

,LONGITUDE,LATITUDE,IDENT,NAME_TXT
0,-176.676024,51.871896,ADK,MOUNT MOFFETT
1,-151.744257,68.136626,AKP,ANAKTUVUK PASS
2,-149.960164,61.167868,TED,ANCHORAGE
3,-149.769204,61.255004,EDF,ELMENDORF
4,-159.597931,61.590169,ANI,ANIAK
...,...,...,...,...
1669,-156.788152,71.273426,BRW,BARROW
1670,-156.781302,71.282300,VIR,BROWERVILLE
1671,-75.164524,43.026523,UCA,UTICA
1672,-90.434470,40.937351,GBG,GALESBURG


In [16]:
# Join the NAVAID dataset with the remaining 49 records
join_uas_nav = pd.merge(non_std1, navaid, on='IDENT', how='left')
join_uas_nav # 32 records were successfully joined using the navaid dataset

,REMARKS,UAS_LOC,IDENT,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,LONGITUDE,LATITUDE,NAME_TXT
0,FACSFAC VACAPES reports PIC reported a silver ...,['6 NM E of RBX'],RBX,5/4/2018,0:06,2018-05-04T00:06Z,PARTY31,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.696939,35.920508,WRIGHT BROTHERS
1,Reported a UAS 500 feet below aircraft while 3...,['3 NM E of OSI'],OSI,5/14/2018,5:46,2018-05-14T05:46Z,ASA330,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-122.281316,37.392505,WOODSIDE
2,Reported a silver and white fixed wing UAS off...,['11 NM SW of OSI'],OSI,5/14/2018,5:53,2018-05-14T05:53Z,N2809E,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-122.281316,37.392505,WOODSIDE
3,PIC reported a black rotary UAS 50 feet below ...,['2 NM SW of PQF'],PQF,5/30/2018,0:31,2018-05-30T00:31Z,XAZTK,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-96.529127,32.809373,MESQUITE
4,After departing RWY 10 PIC reported a UAS belo...,['3 NM NE of TIST'],TIST,5/30/2018,5:00,2018-05-30T05:00Z,KAP249,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PIC reported a black and brown UAS 1-meter in ...,['10 NM NW of CMK'],CMK,6/4/2018,4:51,2018-06-04T04:51Z,DLH403,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-73.581330,41.280102,CARMEL
6,Forest Service Firehelicopter reported UASflew...,['7 NM NNE of DBL'],DBL,6/11/2018,5:53,2018-06-11T05:53Z,NaN,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-106.894691,39.439352,RED TABLE
7,Reported a drone hovering at 600-700 feet 7 NM...,['7 NM N of BAL'],BAL,6/17/2018,8:25,2018-06-17T08:25Z,FOXX2,NCRCC,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.661258,39.171071,BALTIMORE
8,LUKE AFB called and advised of a UAS sighting ...,['20 NM SW of GBN'],GBN,6/19/2018,22:24,2018-06-19T22:24Z,NaN,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-112.674283,32.956263,GILA BEND
9,"Reported NM AC with a black with red UAS, 4 to...",['26 NM SE of JLI'],JLI,6/22/2018,23:22,2018-06-22T23:22Z,RANGR43,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-116.585947,33.140463,JULIAN


In [17]:
# Find the records that actually reference NAVAID
nav_loc = join_uas_nav[join_uas_nav['NAME_TXT'].notnull()]
nav_loc

,REMARKS,UAS_LOC,IDENT,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,LONGITUDE,LATITUDE,NAME_TXT
0,FACSFAC VACAPES reports PIC reported a silver ...,['6 NM E of RBX'],RBX,5/4/2018,0:06,2018-05-04T00:06Z,PARTY31,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.696939,35.920508,WRIGHT BROTHERS
1,Reported a UAS 500 feet below aircraft while 3...,['3 NM E of OSI'],OSI,5/14/2018,5:46,2018-05-14T05:46Z,ASA330,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-122.281316,37.392505,WOODSIDE
2,Reported a silver and white fixed wing UAS off...,['11 NM SW of OSI'],OSI,5/14/2018,5:53,2018-05-14T05:53Z,N2809E,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-122.281316,37.392505,WOODSIDE
3,PIC reported a black rotary UAS 50 feet below ...,['2 NM SW of PQF'],PQF,5/30/2018,0:31,2018-05-30T00:31Z,XAZTK,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-96.529127,32.809373,MESQUITE
5,PIC reported a black and brown UAS 1-meter in ...,['10 NM NW of CMK'],CMK,6/4/2018,4:51,2018-06-04T04:51Z,DLH403,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-73.581330,41.280102,CARMEL
6,Forest Service Firehelicopter reported UASflew...,['7 NM NNE of DBL'],DBL,6/11/2018,5:53,2018-06-11T05:53Z,NaN,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-106.894691,39.439352,RED TABLE
7,Reported a drone hovering at 600-700 feet 7 NM...,['7 NM N of BAL'],BAL,6/17/2018,8:25,2018-06-17T08:25Z,FOXX2,NCRCC,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.661258,39.171071,BALTIMORE
8,LUKE AFB called and advised of a UAS sighting ...,['20 NM SW of GBN'],GBN,6/19/2018,22:24,2018-06-19T22:24Z,NaN,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-112.674283,32.956263,GILA BEND
9,"Reported NM AC with a black with red UAS, 4 to...",['26 NM SE of JLI'],JLI,6/22/2018,23:22,2018-06-22T23:22Z,RANGR43,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-116.585947,33.140463,JULIAN
10,PIC reported a UAS off right side just below A...,['6 NM NE of COL'],COL,6/30/2018,3:31,2018-06-30T03:31Z,RPA4415,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-74.159729,40.311641,COLTS NECK


In [18]:
# Export the 32 records to seperate csv file
nav_loc.to_csv('COMPLETED-non-std-NAVAID.csv', index = False)

In [19]:
nav_loc = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/COMPLETED-non-std-NAVAID.csv')
nav_loc

,REMARKS,UAS_LOC,IDENT,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,LONGITUDE,LATITUDE,NAME_TXT
0,FACSFAC VACAPES reports PIC reported a silver ...,['6 NM E of RBX'],RBX,5/4/2018,0:06,2018-05-04T00:06Z,PARTY31,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.696939,35.920508,WRIGHT BROTHERS
1,Reported a UAS 500 feet below aircraft while 3...,['3 NM E of OSI'],OSI,5/14/2018,5:46,2018-05-14T05:46Z,ASA330,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-122.281316,37.392505,WOODSIDE
2,Reported a silver and white fixed wing UAS off...,['11 NM SW of OSI'],OSI,5/14/2018,5:53,2018-05-14T05:53Z,N2809E,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-122.281316,37.392505,WOODSIDE
3,PIC reported a black rotary UAS 50 feet below ...,['2 NM SW of PQF'],PQF,5/30/2018,0:31,2018-05-30T00:31Z,XAZTK,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-96.529127,32.809373,MESQUITE
4,PIC reported a black and brown UAS 1-meter in ...,['10 NM NW of CMK'],CMK,6/4/2018,4:51,2018-06-04T04:51Z,DLH403,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-73.581330,41.280102,CARMEL
5,Forest Service Firehelicopter reported UASflew...,['7 NM NNE of DBL'],DBL,6/11/2018,5:53,2018-06-11T05:53Z,NaN,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-106.894691,39.439352,RED TABLE
6,Reported a drone hovering at 600-700 feet 7 NM...,['7 NM N of BAL'],BAL,6/17/2018,8:25,2018-06-17T08:25Z,FOXX2,NCRCC,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.661258,39.171071,BALTIMORE
7,LUKE AFB called and advised of a UAS sighting ...,['20 NM SW of GBN'],GBN,6/19/2018,22:24,2018-06-19T22:24Z,NaN,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-112.674283,32.956263,GILA BEND
8,"Reported NM AC with a black with red UAS, 4 to...",['26 NM SE of JLI'],JLI,6/22/2018,23:22,2018-06-22T23:22Z,RANGR43,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-116.585947,33.140463,JULIAN
9,PIC reported a UAS off right side just below A...,['6 NM NE of COL'],COL,6/30/2018,3:31,2018-06-30T03:31Z,RPA4415,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-74.159729,40.311641,COLTS NECK


In [20]:
# Create dictionary for bearing abbreviations and their degrees
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

# Separate distances and bearings into two columns
uas_loc = nav_loc['UAS_LOC']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[2:])
    bearings.append(bearing_abrev)
    

# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(nav_loc['REMARKS'])
new_df['UASLOCATION'] = nav_loc['UAS_LOC']
new_df['IDENT'] = nav_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings
new_df['NAME_TXT'] = nav_loc['NAME_TXT']

new_df

,REMARKS,UASLOCATION,IDENT,Distance,Bearing,NAME_TXT
0,FACSFAC VACAPES reports PIC reported a silver ...,['6 NM E of RBX'],RBX,6,E,WRIGHT BROTHERS
1,Reported a UAS 500 feet below aircraft while 3...,['3 NM E of OSI'],OSI,3,E,WOODSIDE
2,Reported a silver and white fixed wing UAS off...,['11 NM SW of OSI'],OSI,11,SW,WOODSIDE
3,PIC reported a black rotary UAS 50 feet below ...,['2 NM SW of PQF'],PQF,2,SW,MESQUITE
4,PIC reported a black and brown UAS 1-meter in ...,['10 NM NW of CMK'],CMK,10,NW,CARMEL
5,Forest Service Firehelicopter reported UASflew...,['7 NM NNE of DBL'],DBL,7,NNE,RED TABLE
6,Reported a drone hovering at 600-700 feet 7 NM...,['7 NM N of BAL'],BAL,7,N,BALTIMORE
7,LUKE AFB called and advised of a UAS sighting ...,['20 NM SW of GBN'],GBN,20,SW,GILA BEND
8,"Reported NM AC with a black with red UAS, 4 to...",['26 NM SE of JLI'],JLI,26,SE,JULIAN
9,PIC reported a UAS off right side just below A...,['6 NM NE of COL'],COL,6,NE,COLTS NECK


In [21]:
# Convert distances from NM to KM and bearings to degrees
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = pd.to_numeric(new_df['Distance'][i]) * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['NAVAID_Latitude'] = nav_loc['LATITUDE']
new_df['NAVAID_Longitude'] = nav_loc['LONGITUDE']
new_df

,REMARKS,UASLOCATION,IDENT,Distance,Bearing,NAME_TXT,Distance_Kilometers,Bearing_Degrees,NAVAID_Latitude,NAVAID_Longitude
0,FACSFAC VACAPES reports PIC reported a silver ...,['6 NM E of RBX'],RBX,6,E,WRIGHT BROTHERS,11.112,90,35.920508,-75.696939
1,Reported a UAS 500 feet below aircraft while 3...,['3 NM E of OSI'],OSI,3,E,WOODSIDE,5.556,90,37.392505,-122.281316
2,Reported a silver and white fixed wing UAS off...,['11 NM SW of OSI'],OSI,11,SW,WOODSIDE,20.372,225,37.392505,-122.281316
3,PIC reported a black rotary UAS 50 feet below ...,['2 NM SW of PQF'],PQF,2,SW,MESQUITE,3.704,225,32.809373,-96.529127
4,PIC reported a black and brown UAS 1-meter in ...,['10 NM NW of CMK'],CMK,10,NW,CARMEL,18.520,315,41.280102,-73.581330
5,Forest Service Firehelicopter reported UASflew...,['7 NM NNE of DBL'],DBL,7,NNE,RED TABLE,12.964,23,39.439352,-106.894691
6,Reported a drone hovering at 600-700 feet 7 NM...,['7 NM N of BAL'],BAL,7,N,BALTIMORE,12.964,0,39.171071,-76.661258
7,LUKE AFB called and advised of a UAS sighting ...,['20 NM SW of GBN'],GBN,20,SW,GILA BEND,37.040,225,32.956263,-112.674283
8,"Reported NM AC with a black with red UAS, 4 to...",['26 NM SE of JLI'],JLI,26,SE,JULIAN,48.152,135,33.140463,-116.585947
9,PIC reported a UAS off right side just below A...,['6 NM NE of COL'],COL,6,NE,COLTS NECK,11.112,45,40.311641,-74.159729


In [22]:
# Calculate UAS lat/long
uas_lat = []
uas_long = []


for i in range(len(new_df)):
    lat_nav = pd.to_numeric(new_df['NAVAID_Latitude'][i])
    long_nav = pd.to_numeric(new_df['NAVAID_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_nav, long_nav)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)
    
# Append UAS Lat/Long information to DataFrame
new_df['UAS_Latitude'] = uas_lat
new_df['UAS_Longitude'] = uas_long


new_df

,REMARKS,UASLOCATION,IDENT,Distance,Bearing,NAME_TXT,Distance_Kilometers,Bearing_Degrees,NAVAID_Latitude,NAVAID_Longitude,UAS_Latitude,UAS_Longitude
0,FACSFAC VACAPES reports PIC reported a silver ...,['6 NM E of RBX'],RBX,6,E,WRIGHT BROTHERS,11.112,90,35.920508,-75.696939,35.920445,-75.573820
1,Reported a UAS 500 feet below aircraft while 3...,['3 NM E of OSI'],OSI,3,E,WOODSIDE,5.556,90,37.392505,-122.281316,37.392488,-122.218573
2,Reported a silver and white fixed wing UAS off...,['11 NM SW of OSI'],OSI,11,SW,WOODSIDE,20.372,225,37.392505,-122.281316,37.262598,-122.443711
3,PIC reported a black rotary UAS 50 feet below ...,['2 NM SW of PQF'],PQF,2,SW,MESQUITE,3.704,225,32.809373,-96.529127,32.785753,-96.557086
4,PIC reported a black and brown UAS 1-meter in ...,['10 NM NW of CMK'],CMK,10,NW,CARMEL,18.520,315,41.280102,-73.581330,41.397910,-73.737925
5,Forest Service Firehelicopter reported UASflew...,['7 NM NNE of DBL'],DBL,7,NNE,RED TABLE,12.964,23,39.439352,-106.894691,39.546821,-106.835760
6,Reported a drone hovering at 600-700 feet 7 NM...,['7 NM N of BAL'],BAL,7,N,BALTIMORE,12.964,0,39.171071,-76.661258,39.287843,-76.661258
7,LUKE AFB called and advised of a UAS sighting ...,['20 NM SW of GBN'],GBN,20,SW,GILA BEND,37.040,225,32.956263,-112.674283,32.719784,-112.953663
8,"Reported NM AC with a black with red UAS, 4 to...",['26 NM SE of JLI'],JLI,26,SE,JULIAN,48.152,135,33.140463,-116.585947,32.832923,-116.222293
9,PIC reported a UAS off right side just below A...,['6 NM NE of COL'],COL,6,NE,COLTS NECK,11.112,45,40.311641,-74.159729,40.382364,-74.067197


In [56]:
# Export records to csv file
new_df = new_df.drop(['Distance','Bearing','Distance_Kilometers','Bearing_Degrees'], 1)
new_df = new_df.rename({'NAVAID_Longitude': 'Origin_Longitude', 'NAVAID_Latitude': 'Origin_Latitude', 'NAME_TXT': 'NAME'}, axis=1)
new_df = pd.merge(new_df, non_std, on='REMARKS', how='left')
new_df = new_df.drop(['UAS_LOC','IDENT_y','NAME_y','LATITUDE','LONGITUDE','ICAO_ID'], 1)
new_df = new_df.rename({'UASLOCATION': 'UAS_LOC', 'IDENT_x': 'IDENT', 'NAME_x': 'NAME'}, axis=1)
new_df.to_csv('COMPLETED-Non-std-32NAVAID.csv', index = False)

In [ ]:
########################################################################################################################

In [23]:
# Check the unmatched records
len(join_uas_nav[join_uas_nav['NAME_TXT'].isnull()])
# We have 17 remaining records

17

In [24]:
# Export remaining 17 records to csv file
remaining = join_uas_nav[join_uas_nav['NAME_TXT'].isnull()]
remaining.to_csv('non-std_uasLoc_noAir_noNav.csv', index = False)

In [25]:
# Check for runways in the remaining records
rwy = remaining[remaining["IDENT"] == 'RWY']
rwy # We have 8 records 

,REMARKS,UAS_LOC,IDENT,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,LONGITUDE,LATITUDE,NAME_TXT
13,Pilot reported a 4 foot silver UAS orbiting at...,['5 NM S of RWY'],RWY,8/11/2018,0:41,2018-08-11T00:41Z,PICK213,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,Pilot reported a white and orange UAS approxim...,['1 NM WSW of RWY'],RWY,12/8/2018,23:21,2018-12-08T23:21Z,N43RQ,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,While Aircraft was operating in the pattern at...,['5 NM N of RWY'],RWY,12/13/2018,0:44,2018-12-13T00:44Z,N6969M,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,San Diego PD observed a UAS 1.5 NM S of RWY 27...,['5 NM S of RWY'],RWY,11/11/2020,3:01,2020-11-11T03:01Z,NaN,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,Security Guard observed a UAS pass overhead at...,['4 NM E of RWY'],RWY,11/16/2020,16:57,2020-11-16T16:57Z,NaN,DEN,UAS,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,Massachusetts State PD notified the Tower pers...,['5 NM WSW of RWY'],RWY,4/28/2021,21:23,2021-04-28T21:23Z,NaN,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,"ATCT observed a UAS below 500 feet, .5 NM S of...",['5 NM S of RWY'],RWY,6/4/2021,17:50,2021-06-04T17:50Z,NaN,DEN,UAS,MOR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,"N911PX/Helo, observed a green/white quadcopter...",['25 NM W of RWY'],RWY,6/17/2021,18:30,2021-06-17T18:30Z,NaN,JATOC,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Export to csv file and send to Lex to check for IDENTS
rwy.to_csv('Non-std-runways.csv', index = False)

In [27]:
# Check the remaining records
others = remaining[remaining["IDENT"] != 'RWY']
others

,REMARKS,UAS_LOC,IDENT,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,LONGITUDE,LATITUDE,NAME_TXT
4,After departing RWY 10 PIC reported a UAS belo...,['3 NM NE of TIST'],TIST,5/30/2018,5:00,2018-05-30T05:00Z,KAP249,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,"Reported a white quad copter at 1,000 FT 4 N o...",['4 N of MUGU'],MUGU,5/9/2019,5:25,2019-05-09T05:25Z,LIMO42,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,PIC reported a white UAS pass 500 feet belowwh...,['9 NM E of ETXV'],ETXV,6/10/2019,1:30,2019-06-10T01:30Z,N523GC,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,Ground crew observed UAS 2 NM N of ATCT. ATCT ...,['2 NM N of ATCT'],ATCT,11/9/2020,20:32,2020-11-09T20:32Z,NaN,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,LUF airspace manager reported to ZAB aircraft ...,['10 NM NW of TANK'],TANK,12/14/2020,23:04,2020-12-14T23:04Z,ROOK1,DEN,UAS,NOTIFICATION-NONE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,LUF airspace manager reported to ZAB aircraft ...,['13 NM SE of BUGG'],BUGG,12/14/2020,23:55,2020-12-14T23:55Z,DEADLY1,DEN,UAS,NOTIFICATION-NONE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Multiple aircraft observed multiple UAS off th...,['24 NM SW of DIA'],DIA,12/25/2020,3:20,2020-12-25T03:20Z,AIP1813,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,TJIG reports aircraft observed a UAS while orb...,['5 NM W of TJIG'],TJIG,3/14/2021,22:54,2021-03-14T22:54Z,N864BF,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,TJGI reports an aircraft observed a red quad-c...,['3 NM W of TJIG'],TJIG,4/5/2021,18:51,2021-04-05T18:51Z,N4165U,DEN,UAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Export the remaining 9 records to csv file and to be explored next
others.to_csv('Non-std-9others.csv', index = False)